In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.Builder().getOrCreate()

In [21]:
df = spark.read.csv('DataSet_Emails.csv' ,
    header = True ,
    inferSchema = True ,  
    multiLine=True, 
    escape="\"", 
    quote="\"",
    sep=",",
    encoding="UTF-8",
    mode="DROPMALFORMED")

In [22]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+----------+--------------------+--------------------+
|summary|              _c0|        message_id|                text|             label|label_text|             subject|             message|
+-------+-----------------+------------------+--------------------+------------------+----------+--------------------+--------------------+
|  count|            31716|             31716|               31665|             31716|     31716|               31442|               31371|
|   mean|          15857.5|16854.187539412284|                NULL|0.5096165973010468|      NULL|            386429.0|            71403.25|
| stddev|9155.764905238666| 9734.616391716854|                NULL|0.4999153936875302|      NULL|  481877.22667086066|   136659.2035841348|
|    min|                0|                 0| ( b  $ b ! zck...|                 0|       ham|                ( b| & who wants to ...|
|    max|           

In [23]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- message_id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- label_text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- message: string (nullable = true)
 |-- date: date (nullable = true)



In [24]:
df.count()

31716

# Nettoyage du texte


In [25]:
from pyspark.sql.functions import *
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+---+----------+----+-----+----------+-------+-------+----+
|_c0|message_id|text|label|label_text|subject|message|date|
+---+----------+----+-----+----------+-------+-------+----+
|  0|         0|  51|    0|         0|    274|    345|   0|
+---+----------+----+-----+----------+-------+-------+----+



In [32]:
duplicates_count = df.count() - df.dropDuplicates().count()
duplicates_count


0

In [33]:
df = df.dropna(subset=["text"])

In [27]:
from pyspark.sql.functions import col, lower, regexp_replace

df = df.withColumn("text", lower(col("text")))
df = df.withColumn("text", regexp_replace("text", "[^a-zA-Z0-9 ]", " "))

In [28]:
df.show(5)

+-----+----------+--------------------+-----+----------+--------------------+--------------------+----------+
|  _c0|message_id|                text|label|label_text|             subject|             message|      date|
+-----+----------+--------------------+-----+----------+--------------------+--------------------+----------+
|19261|     30704|  zsnb   smart me...|    1|      spam|( zsnb ] smart me...|yo !\nget a huge ...|2004-12-08|
|24256|     20215|  complimentary c...|    1|      spam|- complimentary c...|                NULL|2004-10-29|
|  597|      2828|04   01 assignmen...|    0|       ham|04 / 01 assignmen...|line\nattached is...|2001-03-23|
| 9949|      2194|10   2000   days ...|    0|       ham|10 / 2000 - days ...|the above referen...|2000-12-11|
|25610|     14261|11   30 and 12   ...|    0|       ham|11 / 30 and 12 / ...|for the next few ...|2001-12-05|
+-----+----------+--------------------+-----+----------+--------------------+--------------------+----------+
only showi

# Distribution des labels


In [35]:
df.groupBy('label_text').count().show()

+----------+-----+
|label_text|count|
+----------+-----+
|       ham|14982|
|      spam|13829|
+----------+-----+



# Tokenisation

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
tokenizer = Tokenizer(inputCol="text", outputCol="words")


In [ ]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")


# Stopwords